<a href="https://colab.research.google.com/github/Rishiatweb/Selene/blob/main/LearningDistillation(Torchdistill).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torchdistill torch torchvision

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.0/89.0 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 46.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 32.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 38.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 51.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling

In [2]:
!git clone https://github.com/yoshitomo-matsubara/torchdistill.git
%cd torchdistill

Cloning into 'torchdistill'...
remote: Enumerating objects: 11067, done.
remote: Counting objects: 100% (2030/2030), done.
remote: Compressing objects: 100% (516/516), done.
remote: Total 11067 (delta 1596), reused 1532 (delta 1514), pack-reused 9037 (from 5)
Receiving objects: 100% (11067/11067), 10.81 MiB | 14.54 MiB/s, done.
Resolving deltas: 100% (6778/6778), done.
/content/torchdistill


In [ ]:
!git clone https://github.com/arcee-ai/DistillKit
%cd "set your own path, i am having library conflicts"

In [3]:
!pip install -q "fsspec==2023.9.2"
!pip install -q --upgrade transformers datasets accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.4/173.4 kB 5.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2023.9.2 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.8/494.8 kB 13.2 MB/s eta 0:00:00


In [4]:
from huggingface_hub import notebook_login
notebook_login()

In [7]:
from datasets import load_dataset
dataset = load_dataset("imdb")
train_dataset = dataset["train"].shuffle(seed=42).select(range(10000)) # 10k for training
eval_dataset = dataset["test"].shuffle(seed=42).select(range(1000))  # 1k for evaluation
print(train_dataset[0])

{'text': 'There is no relation at all between Fortier and Profiler but the fact that both are police series about violent crimes. Profiler looks crispy, Fortier looks classic. Profiler plots are quite simple. Fortier\'s plot are far more complicated... Fortier looks more like Prime Suspect, if we have to spot similarities... The main character is weak and weirdo, but have "clairvoyance". People like to compare, to judge, to evaluate. How about just enjoying? Funny thing too, people writing Fortier looks American but, on the other hand, arguing they prefer American series (!!!). Maybe it\'s the language, or the spirit, but I think this series is more English than American. By the way, the actors are really good and funny. The acting is not superficial at all...', 'label': 1}


In [9]:
from transformers import AutoTokenizer
teacher_tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
def preprocess_function(examples):
    return teacher_tokenizer(examples["text"], truncation=True, padding="max_length", max_length=128)
tokenized_train = train_dataset.map(preprocess_function, batched=True)
tokenized_test = eval_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [16]:
from transformers import AutoModelForSequenceClassification, BertConfig, BertForSequenceClassification
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

teacher_model = AutoModelForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels=2
).to(device)

student_config = BertConfig(
    num_hidden_layers=3,
    num_attention_heads=4,
    hidden_size=256,
    intermediate_size=1024,
    num_labels=2
)
student_model = BertForSequenceClassification(config=student_config).to(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [17]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import Trainer

class DistillationTrainer(Trainer):
    def __init__(self, *args, teacher_model=None, alpha=0.5, temperature=2.0, **kwargs):
        super().__init__(*args, **kwargs)
        self.teacher_model = teacher_model
        self.alpha = alpha
        self.temperature = temperature

    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        # Move inputs to the correct device
        inputs = {k: v.to(model.device) for k, v in inputs.items()}

        # 1. Get student's output
        student_outputs = model(**inputs)
        student_loss = student_outputs.loss
        student_logits = student_outputs.logits
        with torch.no_grad():
            teacher_outputs = self.teacher_model(**inputs)
            teacher_logits = teacher_outputs.logits
        distillation_loss = nn.KLDivLoss(reduction="batchmean")(
            F.log_softmax(student_logits / self.temperature, dim=-1),
            F.softmax(teacher_logits / self.temperature, dim=-1)
        ) * (self.temperature ** 2)
        loss = self.alpha * student_loss + (1.0 - self.alpha) * distillation_loss
        return (loss, student_outputs) if return_outputs else loss

In [18]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="distilbert-imdb",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    learning_rate=2e-5,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
)

trainer = DistillationTrainer(
    model=student_model,
    args=training_args,
    teacher_model=teacher_model,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=teacher_tokenizer,
    alpha=0.5,
    temperature=4.0,
)

trainer.train()

/tmp/ipython-input-17-3112123651.py:8: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `DistillationTrainer.__init__`. Use `processing_class` instead.
  super().__init__(*args, **kwargs)


Epoch,Training Loss,Validation Loss
1,0.352800,0.350595
2,0.350100,0.331629
3,0.330700,0.317442


TrainOutput(global_step=1875, training_loss=0.33820669759114586, metrics={'train_runtime': 261.754, 'train_samples_per_second': 114.611, 'train_steps_per_second': 7.163, 'total_flos': 56127697920000.0, 'train_loss': 0.33820669759114586, 'epoch': 3.0})

In [20]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.7 MB/s eta 0:00:00


In [21]:
import evaluate
import numpy as np

accuracy_metric = evaluate.load("accuracy")
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy_metric.compute(predictions=predictions, references=labels)

trainer_with_metrics = DistillationTrainer(
    model=student_model,
    args=training_args,
    teacher_model=teacher_model,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=teacher_tokenizer,
    compute_metrics=compute_metrics,
    alpha=0.5,
    temperature=4.0,
)

print("--- Evaluating the Distilled Student Model ---")
eval_results = trainer_with_metrics.evaluate(eval_dataset=tokenized_test)
print(eval_results)

/tmp/ipython-input-17-3112123651.py:8: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `DistillationTrainer.__init__`. Use `processing_class` instead.
  super().__init__(*args, **kwargs)


--- Evaluating the Distilled Student Model ---


{'eval_loss': 0.3174422085285187, 'eval_model_preparation_time': 0.0016, 'eval_accuracy': 0.71, 'eval_runtime': 7.9253, 'eval_samples_per_second': 126.178, 'eval_steps_per_second': 7.949}


In [22]:
teacher_trainer = Trainer(
    model=teacher_model,
    args=training_args,
    eval_dataset=tokenized_test,
    tokenizer=teacher_tokenizer,
    compute_metrics=compute_metrics
)

print("\n--- Evaluating the Teacher Model ---")
teacher_results = teacher_trainer.evaluate()
print(teacher_results)

/tmp/ipython-input-22-3773931863.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  teacher_trainer = Trainer(



--- Evaluating the Teacher Model ---


{'eval_loss': 0.7127439975738525, 'eval_model_preparation_time': 0.0048, 'eval_accuracy': 0.511, 'eval_runtime': 7.629, 'eval_samples_per_second': 131.078, 'eval_steps_per_second': 8.258}


In [23]:
fresh_student_model = BertForSequenceClassification(config=student_config)

no_distill_trainer = Trainer(
    model=fresh_student_model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=teacher_tokenizer,
    compute_metrics=compute_metrics,
)

print("\n--- Training Student from Scratch (No Distillation) ---")
no_distill_trainer.train()

print("\n--- Evaluating the Student Model (Trained from Scratch) ---")
scratch_results = no_distill_trainer.evaluate()
print(scratch_results)

/tmp/ipython-input-23-2205131627.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  no_distill_trainer = Trainer(



--- Training Student from Scratch (No Distillation) ---


Epoch,Training Loss,Validation Loss,Accuracy
1,0.693600,0.689532,0.627000
2,0.683700,0.568948,0.704000
3,0.566200,0.539773,0.719000



--- Evaluating the Student Model (Trained from Scratch) ---


{'eval_loss': 0.5397733449935913, 'eval_accuracy': 0.719, 'eval_runtime': 0.571, 'eval_samples_per_second': 1751.349, 'eval_steps_per_second': 110.335, 'epoch': 3.0}


In [25]:
print("\n--- Comparison of Results ---")
print(f"Teacher Model Accuracy: {teacher_results['eval_accuracy']:.4f}")
print(f"Distilled Student Model Accuracy: {eval_results['eval_accuracy']:.4f}")
print(f"Student Model (from scratch) Accuracy: {scratch_results['eval_accuracy']:.4f}")


--- Comparison of Results ---
Teacher Model Accuracy: 0.5110
Distilled Student Model Accuracy: 0.7100
Student Model (from scratch) Accuracy: 0.7190


**RESULTS**: It is evident that inspite of using a smaller model trained from a comparatively larger model, one can have similar results if not better, using lesser resources in training from scratch